In [32]:
import requests
import os
from pathlib import Path
import torch
import sys

from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

import torch.nn as nn
import torch.functional as F

In [33]:
SERVER_SPLIT = '172.16.5.71'
PORT_FLASK = 5000
FOLDER = Path(f'{os.getcwd()}/partitions')

def untar_file(file_path, folder_path):
    import zipfile
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(folder_path)

def get_data_loader(client_id, folder_path, batch_size=32):
    full_path = request_local_train_data(client_id, FOLDER)
    from datasets import load_from_disk
    dataset_hug = load_from_disk(full_path)
    #resize of the image before tensor conversion
    class Resize:
        def __init__(self, size):
            self.size = size

        def __call__(self, img):
            img = img.resize(self.size)
            return img

    class Grayscale:
        def __call__(self, img):
            return img.convert("L")

    transforms = ToTensor()
    
    def apply_transforms(batch):
        batch["image"] = [Resize((208, 176))(img) for img in batch["image"]]
        batch["image"] = [Grayscale()(img) for img in batch["image"]]
        batch["image"] = [transforms(img) for img in batch["image"]]
        return batch
    
    partition_torch = dataset_hug.with_transform(apply_transforms)
    # Now, you can check if you didn't make any mistakes by calling partition_torch[0]
    dataloader = DataLoader(partition_torch, batch_size=batch_size, shuffle=True)
    return dataloader

def request_local_train_data(client_id, folder_path):
    url = f'http://{SERVER_SPLIT}:{PORT_FLASK}/partitions/{client_id}'
    response = requests.get(url)
    os.makedirs(f'{folder_path}/{client_id}', exist_ok=True)
    
    with open(f'{folder_path}/{client_id}/local_train-{client_id}.zip', 'wb') as file:
        file.write(response.content)
    
    untar_file(f"{folder_path}/{client_id}/local_train-{client_id}.zip", f"{folder_path}/{client_id}")
    full_path = f'{folder_path}/{client_id}/partition_{client_id}'
    return full_path

In [34]:
class CustomCNN(nn.Module):
    def __init__(self, img_height, img_width, num_classes=4):
        super(CustomCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout2 = nn.Dropout(0.25)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.dropout3 = nn.Dropout(0.25)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear((img_height // 8) * (img_width // 8) * 32, 128)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Initialize the model
img_height = 208
img_width = 176
model = CustomCNN(img_height, img_width)

In [35]:
from datasets import load_dataset

In [36]:
from datasets import load_dataset
from pathlib import Path
dataloader = get_data_loader(8, FOLDER)

In [37]:
dataloader

In [38]:
dataloader.dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 849
})

In [39]:
next(iter(dataloader))

{'image': tensor([[[[0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706],
           [0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706],
           [0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706],
           ...,
           [0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706],
           [0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706],
           [0.0706, 0.0706, 0.0706,  ..., 0.0706, 0.0706, 0.0706]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         

In [70]:
import numpy as np


def extract_central_roi(image, roi_size=0.5):
    """
    Extracts a central square region of interest (ROI) from the image.
    
    Parameters:
        image (numpy array): The input image.
        roi_size (float): The fraction of the image to keep (e.g., 0.5 for 50% center).
    
    Returns:
        numpy array: The cropped central ROI.
    """
    h, w = image.shape[:2]  # Get height and width
    roi_h, roi_w = int(h * roi_size), int(w * roi_size)  # Compute ROI size

    # Define central coordinates
    start_h, start_w = (h - roi_h) // 2, (w - roi_w) // 2
    end_h, end_w = start_h + roi_h, start_w + roi_w

    return image[start_h:end_h, start_w:end_w]  # Crop to ROI

def calculate_snr(image):
    # Calcola la media e la deviazione standard dell'intera immagine
    image = extract_central_roi(image, roi_size=1)  # Estraiamo solo la ROI centrale per calcolare SNR e CNR
    mean_signal = np.mean(image)
    #print(mean_signal)
    std_noise = np.std(image)
    
    #print(std_noise)
    
    # Evita la divisione per zero
    if std_noise == 0:
        return float('inf')  # SNR molto alto se non c'è rumore (poco realistico ma utile per evitare errori)
    
    return mean_signal / std_noise

def calculate_cnr(image):
    # In mancanza di due ROI distinte, possiamo approssimare il CNR usando il contrasto tra media e variazione
    mean_signal = np.mean(image)
    std_noise = np.std(image)
    
    # Qui interpretiamo la deviazione standard come una misura del rumore e il contrasto come differenza dalla media
    return mean_signal / std_noise

# Funzione per calcolare i valori medi aggregati di SNR e CNR per un intero dataset
def calculate_dataset_snr_cnr(dataset):
    total_snr = 0
    total_cnr = 0
    num_images = 0
    print("Dataset", dataset)
    total_dataset = dataset.dataset  # Se il dataset è un DataLoader, accedi al dataset sottostante
    new_dataloader = DataLoader(total_dataset, batch_size=1)  # Creare un DataLoader per accedere ai singoli elementi
    list_snr = []
    
    for _, data in enumerate(new_dataloader, 0):  # Assumendo che il dataset sia nel formato (immagine, etichetta)
        image, = data['image']
        image_array = image.numpy()
        
        # Calcolo di SNR e CNR per l'immagine corrente
        snr = calculate_snr(image_array)
        cnr = calculate_cnr(image_array)
        list_snr.append(snr)
        
        total_snr += snr
        total_cnr += cnr
        num_images += 1
    
    # Calcola la media
    mean_snr = total_snr / num_images
    mean_cnr = total_cnr / num_images
    
    mean_snr = np.median(list_snr)
    #print(list_snr)
    
    return mean_snr, mean_cnr

In [68]:
calculate_dataset_snr_cnr(dataloader)

Dataset <torch.utils.data.dataloader.DataLoader object at 0x725193eec770>
[np.float32(0.8086128), np.float32(0.78308487), np.float32(0.9467151), np.float32(0.74339324), np.float32(1.3845339), np.float32(1.037102), np.float32(1.1529859), np.float32(0.8836178), np.float32(1.0141132), np.float32(1.5075747), np.float32(1.2164006), np.float32(1.1865928), np.float32(1.5620016), np.float32(0.9015775), np.float32(0.69991195), np.float32(0.9300856), np.float32(0.9277104), np.float32(0.8831311), np.float32(0.77741754), np.float32(0.91970545), np.float32(0.7936714), np.float32(0.81334424), np.float32(0.7885284), np.float32(0.8313288), np.float32(1.0341887), np.float32(0.91054446), np.float32(1.0299234), np.float32(0.8381914), np.float32(1.3686894), np.float32(0.84080184), np.float32(0.9306344), np.float32(0.86842453), np.float32(0.96911067), np.float32(0.8792295), np.float32(0.7545922), np.float32(1.1006529), np.float32(1.1768132), np.float32(0.93012446), np.float32(0.94667155), np.float32(0.8600

(np.float32(0.9243945), np.float32(0.9718803))

In [71]:
for i in range(15):
    dataloader = get_data_loader(i, FOLDER)
    print(f"Client {i}:")
    print(calculate_dataset_snr_cnr(dataloader))

Client 0:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251a6404680>
(np.float32(0.9214364), np.float32(0.9657058))
Client 1:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251b33a87a0>
(np.float32(0.9208875), np.float32(0.9611069))
Client 2:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251b4d49370>
(np.float32(0.9213692), np.float32(0.961974))
Client 3:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251b33ab650>
(np.float32(0.9109049), np.float32(0.9580064))
Client 4:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251a6404c80>
(np.float32(0.91962546), np.float32(0.9586101))
Client 5:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x725193ee27e0>
(np.float32(0.9194115), np.float32(0.97103345))
Client 6:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x725193e226f0>
(np.float32(0.91545457), np.float32(0.96059555))
Client 7:
Dataset <torch.utils.data.dataloader.DataLoader object at 0x7251a6407f

In [77]:

def calculate_snr(image):
    """Computes the Signal-to-Noise Ratio (SNR) for an image."""
    mean_signal = np.mean(image)
    std_noise = np.std(image)
    return mean_signal / std_noise if std_noise > 0 else float('inf')

def calculate_penalty_based_on_client_mean(dataset, roi_size=1):
    """
    Computes the penalty for a client based on the fraction of images
    with SNR below the client's own mean SNR.
    
    Parameters:
        dataset (tf.data.Dataset): The dataset containing images.
        roi_size (float): The fraction of the image to consider as the central ROI.
    
    Returns:
        float: The penalty score for the client.
    """
    snr_values = []
    
    # Step 1: Compute SNR for each image
    for data in dataset:
        image = data['image']
        image_array = image.numpy()
        roi = extract_central_roi(image_array, roi_size)
        snr = calculate_snr(roi)
        snr_values.append(snr)
    
    if len(snr_values) == 0:
        return 0  # No images, no penalty

    # Step 2: Compute client's mean SNR
    client_mean_snr = np.mean(snr_values)

    # Step 3: Count images with SNR below the client's mean SNR
    low_quality_count = sum(1 for snr in snr_values if snr < client_mean_snr)

    # Step 4: Compute penalty as fraction of bad images
    penalty = low_quality_count / len(snr_values)

    return penalty 

In [74]:
dataloader = get_data_loader(8, FOLDER)

/home/fcr/client-mri/fed-venv-torch/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


In [75]:
dataloader

In [78]:
for i in range(15):
    dataloader = get_data_loader(i, FOLDER)
    print(f"Client {i}:")
    print(calculate_penalty_based_on_client_mean(dataloader))

/home/fcr/client-mri/fed-venv-torch/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Client 0:
0.6296296296296297
Client 1:
0.5185185185185185
Client 2:
0.5555555555555556
Client 3:
0.5925925925925926
Client 4:
0.5185185185185185
Client 5:
0.48148148148148145
Client 6:
0.5185185185185185
Client 7:
0.48148148148148145
Client 8:
0.5555555555555556
Client 9:
0.4444444444444444
Client 10:
0.5925925925925926
Client 11:
0.37037037037037035
Client 12:
0.6666666666666666
Client 13:
0.5555555555555556
Client 14:
0.48148148148148145
